In [ ]:
import spacy
from spacy import displacy

import requests

In [ ]:
spacy.__version__

In [ ]:
api_url = "http://127.0.0.1:8000"
doc_id = "NCBI_Disease:10192393"

In [ ]:
doc_req = requests.get(f"{api_url}/documents/{doc_id}")
doc_json = doc_req.json()

In [ ]:
def get_matches(doc_json, match_source):
    start_end_labels = []
    for ent in doc_json["entities"]:
        if ent["document_section"] != "title" or ent["source"] != match_source:
            continue
        # print(ent)
        ent_req = requests.get(f"{api_url}/entities/{ent['entity_id']}")
        ent_json = ent_req.json()
        start = ent["start_char"]
        end = ent["end_char"]
        label = ent_json["entity_type"]
        start_end_labels.append([start, end, label])
        start_end_labels = sorted(start_end_labels, key=lambda x: x[0])
    return start_end_labels

In [ ]:
def display_matches(text, labels, title):
    colors = {"DISEASE": "linear-gradient(90deg, #999999, #cccccc)"}
    options = {"ents": ["DISEASE"], "colors": colors}
    ex = [{"text": text,
       "ents": [{"start": x[0], "end": x[1], "label": x[2]} for x in labels],
       "title": title}]
    html = displacy.render(ex, style="ent", manual=True, options=options)
    return html

In [ ]:
doc_text = doc_json["title"]

In [ ]:
m = display_matches(doc_text, get_matches(doc_json, "NCBI_Disease_Train"), "NCBI Disease annotation")

In [ ]:
m = display_matches(doc_text, get_matches(doc_json, "scispacy_en_ner_bc5cdr_md"), "scispacy_en_ner_bc5cdr_md NER")